In [1]:
from transformers import ViTForImageClassification, TrainingArguments, Trainer
from transformers import ViTFeatureExtractor
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from PIL import Image
import json
import os

c:\Users\Shreyas Holkar\deep cnn image classifier\edi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_directory = "E:/Dataset/timeday_dataset"

In [3]:
directory_images_misc = os.path.join(base_directory, "misc")
directory_annotations_misc_labels = os.path.join(base_directory, "misc_labels")

image_files = os.listdir(directory_images_misc)
# Print the total count of image files
print(f"Total number of files in base_directory_images: {len(image_files)}")
# Optionally, print the files themselves (can be omitted if the list is too long)
#print("Files in base_directory_images:", image_files)

txt_files = os.listdir(directory_annotations_misc_labels)
print(f"Total number of files in base_directory_images: {len(txt_files)}")
#print("Files in base_directory_images:", txt_files)

Total number of files in base_directory_images: 4030
Total number of files in base_directory_images: 4030


In [4]:
'''Debugging: Check if filenames are matching
image_basenames = [os.path.splitext(f)[0] for f in image_files]  # Remove extension
json_basenames = [os.path.splitext(f)[0] for f in txt_files]    # Remove extension

# Find missing JSON files
missing_json_files = [f for f in image_basenames if f not in json_basenames]

print(f"Total image basenames: {len(image_basenames)}")
print(f"Total text basenames: {len(json_basenames)}")
print("Missing text annotations:", missing_json_files)'''

'Debugging: Check if filenames are matching\nimage_basenames = [os.path.splitext(f)[0] for f in image_files]  # Remove extension\njson_basenames = [os.path.splitext(f)[0] for f in txt_files]    # Remove extension\n\n# Find missing JSON files\nmissing_json_files = [f for f in image_basenames if f not in json_basenames]\n\nprint(f"Total image basenames: {len(image_basenames)}")\nprint(f"Total text basenames: {len(json_basenames)}")\nprint("Missing text annotations:", missing_json_files)'

In [4]:
import pandas as pd

images_path = "E:/Dataset/timeday_dataset/misc"
annotations_path = "E:/Dataset/timeday_dataset/misc_labels"
output_csv = "misc_labels.csv"

# Initialize list to store data
data = []

# Process each annotation file
for annotation_file in os.listdir(annotations_path):
    if annotation_file.endswith(".txt"):
        image_name = annotation_file.replace(".txt", ".jpg")  # Assuming image is .jpg
        file_path = os.path.join(annotations_path, annotation_file)
        
        with open(file_path, 'r') as f:
            lines = f.readlines()
            if not lines:
                print(f"Warning: {annotation_file} is empty.")
                continue

            # Extract class IDs
            try:
                class_ids = [int(line.split()[0]) for line in lines]
                # Choose label strategy: Most frequent class
                label = max(set(class_ids), key=class_ids.count)
                # Append to data
                data.append([image_name, label])
            except Exception as e:
                print(f"Error processing file {annotation_file}: {e}")

# Check if data is populated
if not data:
    print("No data found. Please check your annotation files and paths.")
else:
    # Save to CSV
    df = pd.DataFrame(data, columns=["image", "label"])
    df.to_csv(output_csv, index=False)
    print(f"Dataset CSV saved to {output_csv}")

Dataset CSV saved to misc_labels.csv


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [6]:
class ImageClassificationDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transform=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transform = transform
        self.data = []
        
        for annotation_file in os.listdir(annotation_dir):
            if annotation_file.endswith(".txt"):
                image_name = annotation_file.replace(".txt", ".jpg")
                image_path = os.path.join(image_dir, image_name)
                file_path = os.path.join(annotation_dir, annotation_file)
                
                if not os.path.exists(image_path):
                    print(f"Skipping missing file: {image_name}")
                    continue
                
                with open(file_path, 'r') as f:
                    lines = f.readlines()
                    if not lines:
                        continue
                    
                    class_ids = [int(line.split()[0]) for line in lines]
                    label = max(set(class_ids), key=class_ids.count)
                    self.data.append((image_name, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_name, label = self.data[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


# Paths
image_dir = "E:/Dataset/timeday_dataset/misc"
annotation_dir = "E:/Dataset/timeday_dataset/misc_labels"

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset and DataLoader
dataset = ImageClassificationDataset(image_dir, annotation_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)



Skipping missing file: 754.jpg
Skipping missing file: 755.jpg
Skipping missing file: 758.jpg
Skipping missing file: 762.jpg
Skipping missing file: 763.jpg
Skipping missing file: 764.jpg
Skipping missing file: 765.jpg
Skipping missing file: 767.jpg
Skipping missing file: IMG_0949.jpg
Skipping missing file: IMG_0950.jpg
Skipping missing file: IMG_0951.jpg
Skipping missing file: IMG_0952.jpg
Skipping missing file: IMG_0953.jpg
Skipping missing file: IMG_0954.jpg
Skipping missing file: IMG_0955.jpg
Skipping missing file: IMG_0956.jpg
Skipping missing file: IMG_0957.jpg
Skipping missing file: IMG_0958.jpg
Skipping missing file: IMG_0960.jpg
Skipping missing file: IMG_0961.jpg
Skipping missing file: IMG_0964.jpg
Skipping missing file: IMG_0965.jpg
Skipping missing file: IMG_0966.jpg
Skipping missing file: IMG_0967.jpg
Skipping missing file: IMG_0968.jpg
Skipping missing file: IMG_0970.jpg
Skipping missing file: IMG_0971.jpg
Skipping missing file: IMG_0972.jpg
Skipping missing file: IMG_0973.

In [15]:
import torch.nn as nn
import torchvision.models as models

# Load pre-trained ResNet18
model = models.resnet18(pretrained=True)

# Modify the final layer for your 15 classes
num_classes = 15
model.fc = nn.Linear(model.fc.in_features, num_classes)

c:\Users\Shreyas Holkar\deep cnn image classifier\edi\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Shreyas Holkar\deep cnn image classifier\edi\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
import torch.optim as optim

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/10], Loss: 0.8226
Epoch [2/10], Loss: 0.6350
Epoch [3/10], Loss: 0.5961
Epoch [4/10], Loss: 0.5602
Epoch [5/10], Loss: 0.5601
Epoch [6/10], Loss: 0.4949
Epoch [7/10], Loss: 0.4243
Epoch [8/10], Loss: 0.3776
Epoch [9/10], Loss: 0.3376
Epoch [10/10], Loss: 0.3158


In [16]:
import torch.optim as optim

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [17]:
"""import os

image_dir = r"E:\Dataset\timeday_dataset\agv_day"  # Replace with your path
annotation_dir = r"E:\Dataset\timeday_dataset\agv_day_labels"  # Replace with your path

missing_files = []
for annotation_file in os.listdir(annotation_dir):
    if annotation_file.endswith(".txt"):
        image_name = annotation_file.replace(".txt", ".jpg")  # Adjust if not .jpg
        image_path = os.path.join(image_dir, image_name)
        if not os.path.exists(image_path):
            missing_files.append(image_name)

if missing_files:
    print(f"Missing image files: {len(missing_files)}")
else:
    print("All image files are present!")"""


<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Shreyas Holkar\AppData\Local\Temp\ipykernel_14216\3197171620.py:1: SyntaxWarning: invalid escape sequence '\D'
  """import os


'import os\n\nimage_dir = r"E:\\Dataset\timeday_dataset\x07gv_day"  # Replace with your path\nannotation_dir = r"E:\\Dataset\timeday_dataset\x07gv_day_labels"  # Replace with your path\n\nmissing_files = []\nfor annotation_file in os.listdir(annotation_dir):\n    if annotation_file.endswith(".txt"):\n        image_name = annotation_file.replace(".txt", ".jpg")  # Adjust if not .jpg\n        image_path = os.path.join(image_dir, image_name)\n        if not os.path.exists(image_path):\n            missing_files.append(image_name)\n\nif missing_files:\n    print(f"Missing image files: {len(missing_files)}")\nelse:\n    print("All image files are present!")'

In [ ]:
from sklearn.metrics import accuracy_score

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")


In [17]:
torch.save(model.state_dict(), "image_classification_model_v3.pth")
print("Model saved successfully!")

Model saved successfully!


In [18]:
model = models.resnet18(pretrained=False)
num_classes = 15
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Load the trained weights
model.load_state_dict(torch.load("image_classification_model.pth"))
model.eval()  # Set the model to evaluation mode

c:\Users\Shreyas Holkar\deep cnn image classifier\edi\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Shreyas Holkar\AppData\Local\Temp\ipykernel_14216\3591867347.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowl

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

# Load class labels from the text file
def load_class_labels(file_path):
    with open(file_path, "r") as f:
        class_labels = [line.strip() for line in f.readlines()]
    return class_labels

class_labels = load_class_labels("classes.txt")


def predict(image_path, model, class_labels):
    image = preprocess_image(image_path)
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)
    predicted_class = class_labels[predicted.item()]  # Use loaded class labels
    return predicted_class


image_path = "E:/Dataset/timeday_dataset/distribution-substation-equipment-1.jpg"  # Replace with your image path

predicted_class = predict(image_path, model, class_labels)
print(f"The image belongs to class: {predicted_class}")


The image belongs to class: Muffle


In [20]:
from collections import Counter

# Analyze class distribution
class_counts = Counter([label for _, label in dataset.data])
print("Class Distribution:")
for cls, count in class_counts.items():
    print(f"Class {cls}: {count} samples")


Class Distribution:
Class 7: 3236 samples
Class 1: 72 samples
Class 12: 54 samples
Class 8: 186 samples
Class 4: 20 samples
Class 5: 1 samples
Class 13: 19 samples
Class 11: 27 samples
Class 10: 41 samples
Class 9: 58 samples
Class 6: 110 samples
Class 2: 8 samples
Class 3: 16 samples


In [21]:
'''import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18

# Load a pre-trained model
model = resnet18(pretrained=True)
model.eval()

# Define the classes (for demonstration, use ImageNet classes)
imagenet_classes = {i: f"Class {i}" for i in range(1000)}

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# GUI Application
class ImageClassifierApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Classifier")
        
        self.label = tk.Label(root, text="Select an image to classify")
        self.label.pack(pady=10)
        
        self.image_label = tk.Label(root)
        self.image_label.pack(pady=10)
        
        self.select_button = tk.Button(root, text="Select Image", command=self.select_image)
        self.select_button.pack(pady=5)
        
        self.classify_button = tk.Button(root, text="Classify Image", command=self.classify_image, state=tk.DISABLED)
        self.classify_button.pack(pady=5)
        
        self.image_path = None
    
    def select_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            self.image_path = file_path
            image = Image.open(file_path)
            image.thumbnail((300, 300))
            img = ImageTk.PhotoImage(image)
            self.image_label.config(image=img)
            self.image_label.image = img
            self.classify_button.config(state=tk.NORMAL)
    
    def classify_image(self):
        if self.image_path:
            image = Image.open(self.image_path)
            input_tensor = transform(image).unsqueeze(0)
            
            with torch.no_grad():
                outputs = model(input_tensor)
                _, predicted = torch.max(outputs, 1)
                class_id = predicted.item()
                class_name = imagenet_classes.get(class_id, "Unknown Class")
            
            messagebox.showinfo("Classification Result", f"Predicted Class: {class_name}")

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = ImageClassifierApp(root)
    root.mainloop()'''


'import tkinter as tk\nfrom tkinter import filedialog, messagebox\nfrom PIL import Image, ImageTk\nimport torch\nimport torchvision.transforms as transforms\nfrom torchvision.models import resnet18\n\n# Load a pre-trained model\nmodel = resnet18(pretrained=True)\nmodel.eval()\n\n# Define the classes (for demonstration, use ImageNet classes)\nimagenet_classes = {i: f"Class {i}" for i in range(1000)}\n\n# Image preprocessing\ntransform = transforms.Compose([\n    transforms.Resize((224, 224)),\n    transforms.ToTensor(),\n    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),\n])\n\n# GUI Application\nclass ImageClassifierApp:\n    def __init__(self, root):\n        self.root = root\n        self.root.title("Image Classifier")\n        \n        self.label = tk.Label(root, text="Select an image to classify")\n        self.label.pack(pady=10)\n        \n        self.image_label = tk.Label(root)\n        self.image_label.pack(pady=10)\n        \n        self.selec

In [22]:
import torch.nn.functional as F
from PIL import Image

model.eval()
with torch.no_grad():
    '''image = Image.open("E:/Dataset/timeday_dataset/Combined-Current-and-Voltage-Transformers.png").convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension'''

    image = Image.open("E:/Dataset/timeday_dataset/distribution-substation-equipment-1.jpg").convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    outputs = model(image)
    probabilities = torch.sigmoid(outputs).squeeze(0)  # Apply sigmoid for multi-label probabilities

    # Threshold to decide which classes are predicted (e.g., > 0.5)
    class_names = [
    "COpen blade disconnect switch", "Closed blade disconnect switch", "Open tandem disconnect switch", "Closed tandem disconnect switch", "Breaker", 
    "Fuse disconnect switch", "Glass disc insulator", "Porcelain pin insulator", "Muffle", "Lightning arrester", 
    "Recloser", "Power transformer", "Current transformer", "Potential transformer", "Tripolar disconnect switch"
    ]

    # Assuming 'predicted' is the output from the model (logits or probabilities)
    predicted = torch.sigmoid(outputs) > 0.5  # Convert to binary predictions for multi-label

    # Get the indices of classes predicted as 1 (positive prediction)
    predicted_indices = torch.where(predicted[0] == 1)[0]  # For a single image

    # Map indices to class names
    predicted_classes = [class_names[idx] for idx in predicted_indices.tolist()]

    # Print the predicted class names
    print("Predicted classes:", predicted_classes)

    predicted_index = [i for i, prob in enumerate(probabilities) if prob > 0.5]
    print("Predicted classes:", predicted_index)


Predicted classes: ['Breaker', 'Porcelain pin insulator', 'Muffle']
Predicted classes: [4, 7, 8]


In [24]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import os
import torch
import torchvision.transforms as transforms
from IPython.display import display, HTML

# GUI within Jupyter Notebook
class ImageClassifierNotebookGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Classification GUI")

        # GUI Elements
        self.label = tk.Label(root, text="Select an image for classification")
        self.label.pack(pady=10)

        self.image_label = tk.Label(root)
        self.image_label.pack(pady=10)

        self.select_button = tk.Button(root, text="Select Image", command=self.select_image)
        self.select_button.pack(pady=5)

        self.classify_button = tk.Button(root, text="Classify Image", command=self.classify_image, state=tk.DISABLED)
        self.classify_button.pack(pady=5)

        self.image_path = None

    def select_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            self.image_path = file_path
            image = Image.open(file_path)
            image.thumbnail((300, 300))
            img = ImageTk.PhotoImage(image)
            self.image_label.config(image=img)
            self.image_label.image = img
            self.classify_button.config(state=tk.NORMAL)

    def classify_image(self):
        if self.image_path:
            try:
                # Image preprocessing (update as per your model requirements)
                transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ])

                image = Image.open(self.image_path).convert('RGB')
                input_tensor = transform(image).unsqueeze(0)

                # Replace 'model' with your actual PyTorch model
                model.eval()  # Ensure the model is in evaluation mode
                with torch.no_grad():
                    outputs = model(input_tensor)
                    probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

                # Get top classes and their probabilities
                top_probabilities, top_indices = torch.topk(probabilities, len(class_names))

                results = []
                for i in range(len(top_indices)):
                    class_id = top_indices[i].item()
                    class_name = class_names[class_id]
                    prob = top_probabilities[i].item()
                    results.append(f"{class_name}: {prob:.4f}")

                # Display the result
                result_message = "\n".join(results)
                messagebox.showinfo("Classification Results", f"Predicted Classes:\n{result_message}")

            except Exception as e:
                messagebox.showerror("Error", f"An error occurred: {e}")

# Run the GUI
if __name__ == "__main__":
    root = tk.Tk()
    app = ImageClassifierNotebookGUI(root)
    root.mainloop()